In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras.models import Model
from keras import models
from keras.optimizers import Adam
import numpy as np
import math
from collections import deque
import os
import time
from datetime import datetime
import h5py
import copy
from state import State_2, UltimateTTT_Move

In [ ]:
Q = {}  # state-action values
Nsa = {}  # number of times certain state-action pair has been visited
Ns = {}   # number of times state has been visited
W = {}  # number of total points collected after taking state action pair
P = {}  # initial predicted probabilities of taking certain actions in state
train_episodes = 100
mcts_search = 5
n_pit_network = 3
playgames_before_training = 2
cpuct = 4
training_epochs = 2
learning_rate = 0.001
save_model_path = 'training_dir'
def get_empty_board():
    return State_2()

def print_board(state):
    firstRow = ""
    secondRow = ""
    thirdRow = ""

    totalBoard = state.blocks
    # Takes each board, saves the rows in a variable, then prints the variables
    for boardIndex in range(len(totalBoard)):
        firstRow = firstRow + "|" + " ".join(map(lambda x: 'X' if x == 1.0 else ('O' if x == -1.0 else ' '), totalBoard[boardIndex][0])) + "|"
        secondRow = secondRow + "|" + " ".join(map(lambda x: 'X' if x == 1.0 else ('O' if x == -1.0 else ' '), totalBoard[boardIndex][1])) + "|"
        thirdRow = thirdRow + "|" + " ".join(map(lambda x: 'X' if x == 1.0 else ('O' if x == -1.0 else ' '), totalBoard[boardIndex][2])) + "|"

        # if 3 boards have been collected, then it prints the boards out and resets the variables (firstRow, secondRow, etc.)
        if boardIndex > 1 and (boardIndex + 1) % 3 == 0:
            print(firstRow)
            print(secondRow)
            print(thirdRow)
            print("---------------------")
            firstRow = ""
            secondRow = ""
            thirdRow = ""

def possiblePos(state, subBoard):
    valid_moves = state.get_valid_moves
    if subBoard == 9:
        return range(81)
    return [(move.index_local_board * 9) + (move.x * 3) + move.y for move in valid_moves]

def move(state, action, player):
    index_local_board = action // 9
    x = (action % 9) // 3
    y = action % 3
    move = UltimateTTT_Move(index_local_board, x, y, player)
    state.act_move(move)
    newsubBoard = (x * 3) + y
    return state, newsubBoard, state.game_over

def fill_winning_boards(state):
    new_board = copy.deepcopy(state)
    for i in range(9):
        if state.global_cells[i] == 1:
            new_board.blocks[i] = np.ones((3, 3))
        elif state.global_cells[i] == -1:
            new_board.blocks[i] = -np.ones((3, 3))
    return new_board

def board_to_array(state):
    board = fill_winning_boards(state)
    array = np.zeros((9, 9))
    for i in range(9):
        for x in range(3):
            for y in range(3):
                array[i // 3 * 3 + x, i % 3 * 3 + y] = board.blocks[i][x, y]
    return array

def mcts(s, current_player, mini_board):
    if mini_board == 9:
        possibleA = range(81)
    else:
        possibleA = possiblePos(s, mini_board)

    sArray = board_to_array(s)
    sTuple = tuple(map(tuple, sArray))

    # Khởi tạo các dictionary nếu chưa tồn tại
    if sTuple not in P:
        P[sTuple] = np.ones(81) / 81  # Giá trị khởi tạo policy (cân bằng)
    if sTuple not in Ns:
        Ns[sTuple] = 0
    for a in range(81):  # Khởi tạo tất cả các action
        if (sTuple, a) not in Q:
            Q[(sTuple, a)] = 0
        if (sTuple, a) not in Nsa:
            Nsa[(sTuple, a)] = 0
        if (sTuple, a) not in W:
            W[(sTuple, a)] = 0

    if len(possibleA) > 0:
        if Ns[sTuple] == 0:  # Tình trạng chưa được mở rộng
            policy, v = nn.predict(sArray.reshape(1, 9, 9))
            v = v[0][0]
            valids = np.zeros(81)
            np.put(valids, possibleA, 1)
            policy = policy.reshape(81) * valids
            policy = policy / np.sum(policy)
            P[sTuple] = policy

            Ns[sTuple] = 1
            return -v

        best_uct = -100
        for a in possibleA:
            uct_a = Q[(sTuple, a)] + cpuct * P[sTuple][a] * (math.sqrt(Ns[sTuple]) / (1 + Nsa[(sTuple, a)]))
            if uct_a > best_uct:
                best_uct = uct_a
                best_a = a

        next_state, mini_board, wonBoard = move(s, best_a, current_player)

        if wonBoard:
            v = 1
        else:
            current_player *= -1
            v = mcts(next_state, current_player, mini_board)
    else:
        return 0

    W[(sTuple, best_a)] += v
    Ns[sTuple] += 1
    Nsa[(sTuple, best_a)] += 1
    Q[(sTuple, best_a)] = W[(sTuple, best_a)] / Nsa[(sTuple, best_a)]
    return -v

def get_action_probs(init_board, current_player, mini_board):
    sArray = board_to_array(init_board)
    sTuple = tuple(map(tuple, sArray))

    # Khởi tạo các dictionary nếu chưa tồn tại
    if sTuple not in P:
        P[sTuple] = np.ones(81) / 81  # Giá trị khởi tạo policy (cân bằng)
    if sTuple not in Ns:
        Ns[sTuple] = 0
    for a in range(81):  # Khởi tạo tất cả các action
        if (sTuple, a) not in Q:
            Q[(sTuple, a)] = 0
        if (sTuple, a) not in Nsa:
            Nsa[(sTuple, a)] = 0

    # Chạy MCTS nhiều lần để lấy xác suất hành động
    for _ in range(mcts_search):
        s = copy.deepcopy(init_board)
        value = mcts(s, current_player, mini_board)

    print("done one iteration of MCTS")

    actions_dict = {}
    for a in possiblePos(init_board, mini_board):
        actions_dict[a] = Nsa[(sTuple, a)] / Ns[sTuple] if Ns[sTuple] > 0 else 0

    action_probs = np.zeros(81)
    for a in actions_dict:
        np.put(action_probs, a, actions_dict[a], mode='raise')

    return action_probs

# def mcts(s, current_player, mini_board):
#     if mini_board == 9:
#         possibleA = range(81)
#     else:
#         possibleA = possiblePos(s, mini_board)

#     sArray = board_to_array(s)
#     sTuple = tuple(map(tuple, sArray))

#     if len(possibleA) > 0:
#         if sTuple not in P.keys():
#             policy, v = nn.predict(sArray.reshape(1, 9, 9))
#             v = v[0][0]
#             valids = np.zeros(81)
#             np.put(valids, possibleA, 1)
#             policy = policy.reshape(81) * valids
#             policy = policy / np.sum(policy)
#             P[sTuple] = policy

#             Ns[sTuple] = 1

#             for a in possibleA:
#                 Q[(sTuple, a)] = 0
#                 Nsa[(sTuple, a)] = 0
#                 W[(sTuple, a)] = 0
#             return -v

#         best_uct = -100
#         for a in possibleA:
#             uct_a = Q[(sTuple, a)] + cpuct * P[sTuple][a] * (math.sqrt(Ns[sTuple]) / (1 + Nsa[(sTuple, a)]))
#             if uct_a > best_uct:
#                 best_uct = uct_a
#                 best_a = a

#         next_state, mini_board, wonBoard = move(s, best_a, current_player)

#         if wonBoard:
#             v = 1
#         else:
#             current_player *= -1
#             v = mcts(next_state, current_player, mini_board)
#     else:
#         return 0

#     W[(sTuple, best_a)] += v
#     Ns[sTuple] += 1
#     Nsa[(sTuple, best_a)] += 1
#     Q[(sTuple, best_a)] = W[(sTuple, best_a)] / Nsa[(sTuple, best_a)]
#     return -v

# def get_action_probs(init_board, current_player, mini_board):
#     for _ in range(mcts_search):
#         s = copy.deepcopy(init_board)
#         value = mcts(s, current_player, mini_board)

#     print("done one iteration of MCTS")

#     actions_dict = {}

#     sArray = board_to_array(init_board)
#     sTuple = tuple(map(tuple, sArray))

#     for a in possiblePos(init_board, mini_board):
#         actions_dict[a] = Nsa[(sTuple, a)] / Ns[sTuple]

#     action_probs = np.zeros(81)

#     for a in actions_dict:
#         np.put(action_probs, a, actions_dict[a], mode='raise')

#     return action_probs

def playgame():
    done = False
    current_player = 1
    game_mem = []
    mini_board = 9

    real_board = get_empty_board()

    while not done:
        s = copy.deepcopy(real_board)
        policy = get_action_probs(s, current_player, mini_board)
        policy = policy / np.sum(policy)
        game_mem.append([board_to_array(real_board), current_player, policy, None])
        action = np.random.choice(len(policy), p=policy)

        print("policy ", policy)
        print("chosen action", action)
        print("mini-board", mini_board)
        print_board(real_board)

        next_state, mini_board, wonBoard = move(real_board, action, current_player)

        if len(possiblePos(next_state, mini_board)) == 0:
            for tup in game_mem:
                tup[3] = 0
            return game_mem

        if wonBoard:
            for tup in game_mem:
                if tup[1] == current_player:
                    tup[3] = 1
                else:
                    tup[3] = -1
            return game_mem

        current_player *= -1
        s = next_state

def neural_network():
    input_layer = layers.Input(shape=(9, 9), name="BoardInput")
    reshape = layers.Reshape((9, 9, 1))(input_layer)
    conv_1 = layers.Conv2D(128, (3, 3), padding='valid', activation='relu', name='conv1')(reshape)
    conv_2 = layers.Conv2D(128, (3, 3), padding='valid', activation='relu', name='conv2')(conv_1)
    conv_3 = layers.Conv2D(128, (3, 3), padding='valid', activation='relu', name='conv3')(conv_2)

    conv_3_flat = layers.Flatten()(conv_3)

    dense_1 = layers.Dense(512, activation='relu', name='dense1')(conv_3_flat)
    dense_2 = layers.Dense(256, activation='relu', name='dense2')(dense_1)

    pi = layers.Dense(81, activation="softmax", name='pi')(dense_2)
    v = layers.Dense(1, activation="tanh", name='value')(dense_2)

    model = Model(inputs=input_layer, outputs=[pi, v])
    model.compile(loss=['categorical_crossentropy', 'mean_squared_error'], optimizer=Adam(learning_rate))

    model.summary()
    return model

def train_nn(nn, game_mem):
    print("Training Network")
    print("length of game_mem", len(game_mem))

    state = []
    policy = []
    value = []

    for mem in game_mem:
        state.append(mem[0])
        policy.append(mem[2])
        value.append(mem[3])

    state = np.array(state)
    policy = np.array(policy)
    value = np.array(value)

    history = nn.fit(state, [policy, value], batch_size=32, epochs=training_epochs, verbose=1)

def pit(nn, new_nn):
    print("Pitting networks")
    nn_wins = 0
    new_nn_wins = 0

    for _ in range(n_pit_network):
        s = get_empty_board()
        mini_board = 9

        while True:
            policy, v = nn.predict(board_to_array(s).reshape(1, 9, 9))
            valids = np.zeros(81)

            possibleA = possiblePos(s, mini_board)

            if len(possibleA) == 0:
                break

            np.put(valids, possibleA, 1)
            policy = policy.reshape(81) * valids
            policy = policy / np.sum(policy)
            action = np.argmax(policy)

            next_state, mini_board, win = move(s, action, 1)
            s = next_state

            if win:
                nn_wins += 1
                break

            policy, v = new_nn.predict(board_to_array(s).reshape(1, 9, 9))
            valids = np.zeros(81)

            possibleA = possiblePos(s, mini_board)

            if len(possibleA) == 0:
                break

            np.put(valids, possibleA, 1)
            policy = policy.reshape(81) * valids
            policy = policy / np.sum(policy)
            action = np.argmax(policy)

            next_state, mini_board, win = move(s, action, -1)
            s = next_state

            if win:
                new_nn_wins += 1
                break

    if (new_nn_wins + nn_wins) == 0:
        print("The game was a complete tie")
        now = datetime.utcnow()
        filename = 'tictactoeTie{}.h5'.format(now)
        model_path = os.path.join(save_model_path, filename)
        nn.save(model_path)
        return False

    win_percent = float(new_nn_wins) / float(new_nn_wins + nn_wins)
    if win_percent > 0.52:
        print("The new network won")
        print(win_percent)
        return True
    else:
        print("The new network lost")
        print(new_nn_wins)
        return False
output_dir = '/kaggle/working'
nn = neural_network()

# def train():
#     global nn
#     global Q
#     global Nsa
#     global Ns
#     global W
#     global P

#     game_mem = []

#     for episode in range(train_episodes):
#         nn.save('temp.h5')
#         old_nn = models.load_model('temp.h5')

#         for _ in range(playgames_before_training):
#             game_mem += playgame()

#         train_nn(nn, game_mem)
#         game_mem = []
#         if pit(old_nn, nn):
#             del old_nn
#             Q = {}
#             Nsa = {}
#             Ns = {}
#             W = {}
#             P = {}
#         else:
#             nn = old_nn
#             del old_nn

#     now = datetime.utcnow()
#     filename = 'tictactoe_MCTS200{}.h5'.format(now)
#     model_path = os.path.join(save_model_path, filename)
#     nn.save(model_path)

In [ ]:
from keras.optimizers import Adam
from keras.models import load_model
import os
import datetime

def save_model(nn, epoch):
    """
    Lưu mô hình sau mỗi epoch
    Args:
        nn: Mô hình Keras hiện tại
        epoch: số epoch hiện tại
    """
    now = datetime.datetime.utcnow()
    filename = f'tictactoe_model_epoch_{epoch}_{now.strftime("%Y%m%d%H%M%S")}.h5'
    model_path = os.path.join(output_dir, filename)
    nn.save(model_path)
    print(f'Model saved to {model_path}')
    

def load_previous_model(output_dir, learning_rate):
    """
    Tải mô hình đã lưu (nếu có), và tái biên dịch với optimizer mới.
    """
    predefined_model_path = '/kaggle/input/ver3/keras/default/1/tictactoe_MCTS200_20241227214414.h5'
    try:
        # Kiểm tra thư mục /kaggle/working/training_dir trước
        if os.path.exists(output_dir):
            model_files = [f for f in os.listdir(output_dir) if f.endswith('.h5')]
            if model_files:
                latest_model = max(model_files, key=lambda f: os.path.getctime(os.path.join(output_dir, f)))
                model_path = os.path.join(output_dir, latest_model)
                print(f'Loading model from {model_path}')
                model = load_model(model_path)
    
                # Tạo lại optimizer và biên dịch lại mô hình
                optimizer = Adam(learning_rate=learning_rate)
                model.compile(optimizer=optimizer, loss=['categorical_crossentropy', 'mean_squared_error'])
                return model

        # Nếu không có mô hình trong output_dir, kiểm tra file cố định trong input
        if os.path.exists(predefined_model_path):
            print(f'Loading model from {predefined_model_path}')
            model = load_model(predefined_model_path)

            # Tạo lại optimizer và biên dịch lại mô hình
            optimizer = Adam(learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss=['categorical_crossentropy', 'mean_squared_error'])
            return model

        # Nếu không tìm thấy mô hình ở bất kỳ đâu, trả về None
        print('No model found in both training_dir and input. Starting from scratch.')
        return None
    except Exception as e:
        print(f'Error loading model: {e}')
        return None
# Hàm huấn luyện với cơ chế lưu mô hình
# def train():
#     global nn
#     global Q
#     global Nsa
#     global Ns
#     global W
#     global P

#     game_mem = []

#     # Tải mô hình đã lưu nếu có, nếu không tạo mô hình mới
#     nn = load_previous_model(output_dir, learning_rate) if load_previous_model(output_dir, learning_rate) is not None else neural_network()

#     for episode in range(train_episodes):
        
#         # Lưu mô hình sau mỗi epoch
#         if episode % 1 == 0:  # Bạn có thể thay đổi tần suất lưu mô hình
#             save_model(nn, episode)

#         nn.save('temp.h5')
#         old_nn = load_model('temp.h5')  # Sử dụng load_model để tải lại mô hình

#         # Tạo optimizer mới cho mô hình tải lại
#         optimizer = Adam(learning_rate=learning_rate)
#         old_nn.compile(optimizer=optimizer, loss=['categorical_crossentropy', 'mean_squared_error'])

#         for _ in range(playgames_before_training):
#             game_mem += playgame()

#         train_nn(nn, game_mem)
#         game_mem = []
        
#         if pit(old_nn, nn):
#             del old_nn
#             Q = {}
#             Nsa = {}
#             Ns = {}
#             W = {}
#             P = {}
#         else:
#             nn = old_nn
#             del old_nn

#     # Lưu mô hình cuối cùng
#     now = datetime.datetime.utcnow()
#     filename = f'tictactoe_MCTS200_{now.strftime("%Y%m%d%H%M%S")}.h5'
#     model_path = os.path.join(save_model_path, filename)
#     nn.save(model_path)
#     print(f'Final model saved to {model_path}')

def train():
    global nn
    global Q
    global Nsa
    global Ns
    global W
    global P

    game_mem = []
    # output_dir = '/kaggle/working/training_dir'
    output_dir = '/kaggle/working'
    # save_model_path = '/kaggle/working/training_dir'
    # Tải mô hình đã lưu nếu có, nếu không tạo mô hình mới
    nn = load_previous_model(output_dir, learning_rate) if load_previous_model(output_dir, learning_rate) is not None else neural_network()

    for episode in range(train_episodes):
        
        # Lưu mô hình sau mỗi epoch
        if episode % 1 == 0:  # Bạn có thể thay đổi tần suất lưu mô hình
            save_model(nn, episode)

        for _ in range(playgames_before_training):
            game_mem += playgame()

        train_nn(nn, game_mem)
        game_mem = []

        if episode % 10 == 0:  # Ví dụ chỉ thay đổi mô hình sau mỗi 10 epoch
            save_model(nn, episode)
    
    # Lưu mô hình cuối cùng
    now = datetime.datetime.utcnow()
    filename = f'tictactoe_MCTS200_{now.strftime("%Y%m%d%H%M%S")}.h5'
    model_path = os.path.join(save_model_path, filename)
    nn.save(model_path)
    print(f'Final model saved to {model_path}')

In [ ]:
train()